In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import os
import shutil

import matplotlib.pyplot as plt
import numpy as np
import time
import pandas as pd
import json
import matplotlib.image as mpimg

import seaborn as sb
import torch
from torch import nn
from torch import optim

import torchvision.transforms.functional as TF
from torchvision import datasets, transforms, models
from collections import OrderedDict

import h5py

from PIL import Image



以下操作全部耗时可能接近1小时，请谨慎操作。不会大量占用本地存储空间，请放心。

In [ ]:
#将训练集图片分为猫和狗，将测试集图片分文件夹放置
train_filenames = os.listdir('train')
train_cat = list(filter(lambda x:x[:3] == 'cat', train_filenames))
train_dog = list(filter(lambda x:x[:3] == 'dog', train_filenames))

def rmrf_mkdir(dirname):
    if os.path.exists(dirname):
        shutil.rmtree(dirname)
    os.mkdir(dirname)

rmrf_mkdir('train2')
os.mkdir('train2\\cat')
os.mkdir('train2\\dog')


for filename in train_cat:
    os.symlink('E:\\AI\\MLND_CN_P7_DogvsCat\\train\\'+filename, 'train2\\cat\\'+filename)

for filename in train_dog:
    os.symlink('E:\\AI\\MLND_CN_P7_DogvsCat\\train\\'+filename, 'train2\\dog\\'+filename)

    
test_filenames = os.listdir('test')
test_filenames_no_jpg = list([x[:-4] for x in test_filenames])


rmrf_mkdir('test2')
for filename_no_jpg, filename in zip(test_filenames_no_jpg, test_filenames):
    rmrf_mkdir('test2\\' + filename_no_jpg)

    os.symlink('E:\\AI\\MLND_CN_P7_DogvsCat\\test\\'+filename, 'test2\\' + filename_no_jpg + '\\' + filename)

In [ ]:
#首先定义一个与训练过的模型，并冻结参数。
model_detect = models.resnet152(pretrained=True)
for p in model_detect.parameters():
    p.requires_grad = False

In [ ]:
detect_transforms = transforms.Compose([transforms.Resize(int(255)),
                                           transforms.CenterCrop(224),
                                           transforms.ToTensor(),
                                           transforms.Normalize([0.485, 0.456, 0.406], 
                                                                [0.229, 0.224, 0.225])])

In [ ]:
#定义一个预测函数，得出1000个输出值，并在下一个函数进行检测
def detect(model, loader, single=False):
    model.cuda()
    model.eval()
    with torch.no_grad():
        outputs = []
        if not single:
            data, labels = loader
            data, labels = data.cuda(), labels.cuda()
        else:
            data = loader.cuda()
        outputs += model(data).cpu().int().numpy().tolist()
        return outputs

In [ ]:
#找出猫狗对应的位置的列表。
pretrain_list ='n01440764 鱼\nn01443537 鱼\nn01484850 鱼\nn01491361 鱼\nn01494475 鱼\nn01496331 鱼\nn01498041 鱼\nn01514668 鸡\nn01514859 鸡\nn01518878 鸵鸟\nn01530575 鸟\nn01531178 鸟\nn01532829 鸟\nn01534433 鸟\nn01537544 鸟\nn01558993 鸟\nn01560419 鸟\nn01580077 鸟\nn01582220 鸟\nn01592084 鸟\nn01601694 鸟\nn01608432 鸟\nn01614925 鹰\nn01616318 鹰\nn01622779 猫头鹰\nn01629819 壁虎\nn01630670 壁虎\nn01631663 壁虎\nn01632458 壁虎\nn01632777 壁虎\nn01641577 蛤蟆\nn01644373 青蛙\nn01644900 青蛙\nn01664065 龟\nn01665541 龟\nn01667114 龟\nn01667778 龟\nn01669191 龟\nn01675722 蜥蜴\nn01677366 蜥蜴\nn01682714 蜥蜴\nn01685808 蜥蜴\nn01687978 蜥蜴\nn01688243 蜥蜴\nn01689811 蜥蜴\nn01692333 蜥蜴\nn01693334 蜥蜴\nn01694178 蜥蜴\nn01695060 蜥蜴\nn01697457 鳄鱼\nn01698640 鳄鱼\nn01704323 恐龙\nn01728572 蛇\nn01728920 蛇\nn01729322 蛇\nn01729977 蛇\nn01734418 蛇\nn01735189 蛇\nn01737021 蛇\nn01739381 蛇\nn01740131 蛇\nn01742172 蛇\nn01744401 蛇\nn01748264 蛇\nn01749939 蛇\nn01751748 蛇\nn01753488 蛇\nn01755581 蛇\nn01756291 蛇\nn01768244 化石\nn01770081 蜘蛛\nn01770393 蝎子\nn01773157 蜘蛛\nn01773549 蜘蛛\nn01773797 蜘蛛\nn01774384 蜘蛛\nn01774750 蜘蛛\nn01775062 蜘蛛\nn01776313 蜘蛛\nn01784675 蜈蚣\nn01795545 鸟\nn01796340 鸟\nn01797886 鸟\nn01798484 鸟\nn01806143 孔雀\nn01806567 鸟\nn01807496 鸟\nn01817953 鸟\nn01818515 鸟\nn01819313 鸟\nn01820546 鸟\nn01824575 鸟\nn01828970 鸟\nn01829413 鸟\nn01833805 鸟\nn01843065 鸟\nn01843383 鸟\nn01847000 鸭子\nn01855032 鹅\nn01855672 鹅\nn01860187 鹅\nn01871265 大象\nn01872401 刺猬\nn01873310 鸭嘴兽\nn01877812 袋鼠\nn01882714 考拉\nn01883070 土拨鼠\nn01910747 水母\nn01914609 珊瑚\nn01917289 珊瑚\nn01924916 海洋生物\nn01930112 海蛇\nn01943899 海螺\nn01944390 蜗牛\nn01945685 蜗牛\nn01950731 海洋生物\nn01955084 海洋生物\nn01968897 海螺\nn01978287 螃蟹\nn01978455 螃蟹\nn01980166 螃蟹\nn01981276 螃蟹\nn01983481 龙虾\nn01984695 龙虾\nn01985128 龙虾\nn01986214 寄居蟹\nn01990800 海洋生物\nn02002556 鸟\nn02002724 鸟\nn02006656 鸟\nn02007558 鸟\nn02009229 鸟\nn02009912 鸟\nn02011460 鸟\nn02012849 鸟\nn02013706 鸟\nn02017213 鸟\nn02018207 鸟\nn02018795 鸟\nn02025239 鸟\nn02027492 鸟\nn02028035 鸟\nn02033041 鸟\nn02037110 鸟\nn02051845 鸟\nn02056570 企鹅\nn02058221 鸟\nn02066245 鲸鱼\nn02071294 鲸鱼\nn02074367 海象\nn02077923 海狮\nn02085620 狗\nn02085782 狗\nn02085936 狗\nn02086079 狗\nn02086240 狗\nn02086646 狗\nn02086910 狗\nn02087046 狗\nn02087394 狗\nn02088094 狗\nn02088238 狗\nn02088364 狗\nn02088466 狗\nn02088632 狗\nn02089078 狗\nn02089867 狗\nn02089973 狗\nn02090379 狗\nn02090622 狗\nn02090721 狗\nn02091032 狗\nn02091134 狗\nn02091244 狗\nn02091467 狗\nn02091635 狗\nn02091831 狗\nn02092002 狗\nn02092339 狗\nn02093256 狗\nn02093428 狗\nn02093647 狗\nn02093754 狗\nn02093859 狗\nn02093991 狗\nn02094114 狗\nn02094258 狗\nn02094433 狗\nn02095314 狗\nn02095570 狗\nn02095889 狗\nn02096051 狗\nn02096177 狗\nn02096294 狗\nn02096437 狗\nn02096585 狗\nn02097047 狗\nn02097130 狗\nn02097209 狗\nn02097298 狗\nn02097474 狗\nn02097658 狗\nn02098105 狗\nn02098286 狗\nn02098413 狗\nn02099267 狗\nn02099429 狗\nn02099601 狗\nn02099712 狗\nn02099849 狗\nn02100236 狗\nn02100583 狗\nn02100735 狗\nn02100877 狗\nn02101006 狗\nn02101388 狗\nn02101556 狗\nn02102040 狗\nn02102177 狗\nn02102318 狗\nn02102480 狗\nn02102973 狗\nn02104029 狗\nn02104365 狗\nn02105056 狗\nn02105162 狗\nn02105251 狗\nn02105412 狗\nn02105505 狗\nn02105641 狗\nn02105855 狗\nn02106030 狗\nn02106166 狗\nn02106382 狗\nn02106550 狗\nn02106662 狗\nn02107142 狗\nn02107312 狗\nn02107574 狗\nn02107683 狗\nn02107908 狗\nn02108000 狗\nn02108089 狗\nn02108422 狗\nn02108551 狗\nn02108915 狗\nn02109047 狗\nn02109525 狗\nn02109961 狗\nn02110063 狗\nn02110185 狗\nn02110341 狗\nn02110627 狗\nn02110806 狗\nn02110958 狗\nn02111129 狗\nn02111277 狗\nn02111500 狗\nn02111889 狗\nn02112018 狗\nn02112137 狗\nn02112350 狗\nn02112706 狗\nn02113023 狗\nn02113186 狗\nn02113624 狗\nn02113712 狗\nn02113799 狗\nn02113978 狗\nn02114367 狼\nn02114548 狼\nn02114712 狼\nn02114855 狼\nn02115641 狼\nn02115913 狼\nn02116738 狼\nn02117135 狼\nn02119022 狐狸\nn02119789 狐狸\nn02120079 狐狸\nn02120505 狐狸\nn02123045 猫\nn02123159 猫\nn02123394 猫\nn02123597 猫\nn02124075 猫\nn02125311 猫\nn02127052 猫\nn02128385 豹\nn02128757 豹\nn02128925 豹\nn02129165 狮子\nn02129604 老虎\nn02130308 豹\nn02132136 熊\nn02133161 熊\nn02134084 熊\nn02134418 熊\nn02137549 猫鼬\nn02138441 猫鼬\nn02165105 昆虫\nn02165456 昆虫\nn02167151 昆虫\nn02168699 昆虫\nn02169497 昆虫\nn02172182 昆虫\nn02174001 昆虫\nn02177972 昆虫\nn02190166 昆虫\nn02206856 昆虫\nn02219486 昆虫\nn02226429 昆虫\nn02229544 昆虫\nn02231487 昆虫\nn02233338 昆虫\nn02236044 昆虫\nn02256656 昆虫\nn02259212 昆虫\nn02264363 昆虫\nn02268443 蜻蜓\nn02268853 蜻蜓\nn02276258 蝴蝶\nn02277742 蝴蝶\nn02279972 蝴蝶\nn02280649 蝴蝶\nn02281406 蝴蝶\nn02281787 蝴蝶\nn02317335 海星\nn02319095 海胆\nn02321529 海洋生物\nn02325366 兔子\nn02326432 兔子\nn02328150 兔子\nn02342885 鼠\nn02346627 鼠\nn02356798 松鼠\nn02361337 鼠\nn02363005 鼠\nn02364673 鼠\nn02389026 马\nn02391049 斑马\nn02395406 猪\nn02396427 猪\nn02397096 猪\nn02398521 河马\nn02403003 牛\nn02408429 牛\nn02410509 牛\nn02412080 羊\nn02415577 羊\nn02417914 羊\nn02422106 羊\nn02422699 羊\nn02423022 羊\nn02437312 骆驼\nn02437616 羊驼\nn02441942 狸\nn02442845 狸\nn02443114 狸\nn02443484 狸\nn02444819 狸\nn02445715 狸\nn02447366 狸\nn02454379 穿山甲\nn02457408 树懒\nn02480495 狒狒\nn02480855 猩猩\nn02481823 猴子\nn02483362 猴子\nn02483708 猴子\nn02484975 猴子\nn02486261 猴子\nn02486410 猴子\nn02487347 猴子\nn02488291 猴子\nn02488702 猴子\nn02489166 猴子\nn02490219 猴子\nn02492035 猴子\nn02492660 猴子\nn02493509 猴子\nn02493793 猴子\nn02494079 猴子\nn02497673 猴子\nn02500267 猴子\nn02504013 大象\nn02504458 大象\nn02509815 浣熊\nn02510455 熊猫\nn02514041 鱼\nn02526121 鱼\nn02536864 鱼\nn02606052 鱼\nn02607072 鱼\nn02640242 鱼\nn02641379 鱼\nn02643566 鱼\nn02655020 鱼\nn02666196 算盘\nn02667093 穆斯林\nn02669723 学士服\nn02672831 手风琴\nn02676566 吉他\nn02687172 航空母舰\nn02690373 飞机\nn02692877 飞艇\nn02699494 教堂\nn02701002 救护车 check\nn02704792 水陆两用车 check\nn02708093 钟\nn02727426 箱子\nn02730930 围裙\nn02747177 垃圾箱\nn02749479 枪\nn02769748 背包\nn02776631 面包柜\nn02777292 体操\nn02782093 热气球\nn02783161 钢笔\nn02786058 邦迪\nn02787622 乐器\nn02788148 楼梯\nn02790996 杠铃\nn02791124 座椅\nn02791270 理发\nn02793495 木屋\nn02794156 表\nn02795169 酒桶\nn02797295 手推车\nn02799071 棒球\nn02802426 篮球\nn02804414 婴儿\nn02804610 乐器\nn02807133 游泳\nn02808304 婴儿毛巾\nn02808440 浴缸\nn02814533 轿车 check\nn02814860 灯塔\nn02815834 烧杯\nn02817516 守卫\nn02823428 啤酒\nn02823750 啤酒\nn02825657 建筑\nn02834397 围兜\nn02835271 双人自行车\nn02837789 比基尼\nn02840245 笔记本\nn02841315 望远镜\nn02843684 信箱\nn02859443 小屋\nn02860847 雪橇\nn02865351 项链\nn02869837 帽子\nn02870880 书柜\nn02871525 书店\nn02877765 瓶盖\nn02879718 弓箭\nn02883205 领结\nn02892201 墓碑\nn02892767 胸罩\nn02894605 海岸\nn02895154 盔甲\nn02906734 扫帚\nn02909870 水桶\nn02910353 皮带\nn02916936 防弹背心\nn02917067 火车\nn02927161 肉铺\nn02930766 出租车 check\nn02939185 锅\nn02948072 蜡烛\nn02950826 炮\nn02951358 艇\nn02951585 订书机\nn02963159 毛衣\nn02965783 反光镜\nn02966193 旋转木马\nn02966687 工具箱\nn02971356 盒子\nn02974003 轮胎\nn02977058 取款机\nn02978881 磁带\nn02979186 磁带\nn02980441 城堡\nn02981792 帆船\nn02988304 cd播放器\nn02992211 大提琴\nn02992529 手机\nn02999410 铁链\nn03000134 铁丝网\nn03000247 铁丝网\nn03000684 电锯\nn03014705 木箱\nn03016953 木柜\nn03017168 锣鼓\nn03018349 柜子\nn03026506 袜子\nn03028079 教堂\nn03032252 剧院\nn03041632 刀\nn03042490 堡垒\nn03045698 斗篷\nn03047690 鞋子\nn03062245 瓶子\nn03063599 杯子\nn03063689 壶\nn03065424 螺旋\nn03075370 锁\nn03085013 键盘\nn03089624 糖果\nn03095699 船\nn03100240 轿车 check\nn03109150 开瓶器\nn03110669 号（乐器）\nn03124043 靴子\nn03124170 帽子\nn03125729 婴儿床\nn03126707 起重机\nn03127747 头盔\nn03127925 木箱\nn03131574 婴儿床\nn03133878 电饭锅\nn03134739 推球\nn03141823 拐杖\nn03146219 盔甲\nn03160309 水库\nn03179701 办公桌\nn03180011 电脑\nn03187595 电话机\nn03188531 尿布\nn03196217 闹钟\nn03197337 手表\nn03201208 餐桌\nn03207743 垫子\nn03207941 洗碗柜\nn03208938 车轮\nn03216828 港口\nn03218198 雪橇\nn03220513 穹顶\nn03223299 地毯\nn03240683 油田\nn03249569 鼓\nn03250847 鼓棒\nn03255030 哑铃\nn03259280 锅\nn03271574 风扇\nn03272010 电吉他\nn03272562 火车 check\nn03290653 电视机\nn03291819 信\nn03297495 咖啡机\nn03314780 化妆品\nn03325584 绒毛\nn03337140 柜子\nn03344393 喷泉\nn03345487 消防车 check\nn03347037 壁炉\nn03355925 旗杆\nn03372029 笛子\nn03376595 座椅\nn03379051 橄榄球\nn03384352 叉车\nn03388043 喷泉\nn03388183 钢笔\nn03388549 床\nn03393912 火车\nn03394916 圆号\nn03400231 平底锅\nn03404251 裘皮\nn03417042 卡车 check\nn03424325 面具\nn03425413 加油\nn03443371 酒杯\nn03444034 卡丁车\nn03445777 高尔夫\nn03445924 高尔夫车\nn03447447 小船\nn03447721 锣鼓\nn03450230 婚纱\nn03452741 钢琴\nn03457902 大棚\nn03459775 轿车 车标 check\nn03461385 菜场\nn03467068 断头台\nn03476684 发饰\nn03476991 发蜡\nn03478589 坦克\nn03481172 榔头\nn03482405 竹筒\nn03483316 吹风机\nn03485407 pos机\nn03485794 手帕\nn03492542 硬盘\nn03494278 口风琴\nn03495258 竖琴\nn03496892 起重机\nn03498962 斧头\nn03527444 手枪\nn03529860 电视机\nn03530642 蜂巢\nn03532672 钩子\nn03534580 裙子\nn03535780 体操\nn03538406 马车\nn03544143 沙漏\nn03584254 音乐播放器\nn03584829 电熨斗\nn03590841 南瓜灯\nn03594734 牛仔裤\nn03594945 吉普车 check\nn03595614 T恤\nn03598930 拼图\nn03599486 黄包车\nn03602883 操纵杆\nn03617480 和服\nn03623198 护具\nn03627232 绳结\nn03630383 医生\nn03633091 勺子\nn03637318 灯\nn03642806 笔记本电脑\nn03649909 割草机\nn03657121 镜头盖\nn03658185 小刀\nn03661043 图书馆\nn03662601 救生船\nn03666591 打火机\nn03670208 加长车 check\nn03673027 轮船\nn03676483 口红\nn03680355 鞋子\nn03690938 护肤品\nn03691459 音响\nn03692522 放大镜\nn03697007 原木\nn03706229 指南针\nn03709823 包\nn03710193 邮箱\nn03710637 泳衣\nn03710721 泳衣\nn03717622 窨井盖\nn03720891 手摇铃\nn03721384 木琴\nn03724870 面具\nn03729826 火柴\nn03733131 绳子\nn03733281 迷宫\nn03733805 烧杯\nn03742115 冰箱\nn03743016 石柱\nn03759954 话筒\nn03761084 微波炉\nn03763968 军人\nn03764736 水壶\nn03769881 小客车 check\nn03770439 短裙\nn03770679 面包车 check\nn03773504 导弹\nn03775071 手套\nn03775546 碗\nn03776460 房车\nn03777568 老爷车\nn03777754 路由器\nn03781244 建筑\nn03782006 显示器\nn03785016 摩托车 check\nn03786901 砚\nn03787032 学士帽\nn03788195 建筑\nn03788365 蚊帐\nn03791053 助动车 check\nn03792782 自行车 check\nn03792972 帐篷\nn03793489 键盘鼠标\nn03794056 捕鼠夹\nn03796401 货车 卡车 check\nn03803284 狗嘴套\nn03804744 钉子\nn03814639 颈托\nn03814906 项链\nn03825788 奶瓶\nn03832673 笔记本电脑\nn03837869 建筑\nn03838899 黑管\nn03840681 埙\nn03841143 仪表盘\nn03843555 机油滤清器\nn03854065 管风琴\nn03857828 示波器\nn03866082 礼服\nn03868242 牛车\nn03868863 呼吸器\nn03871628 零食\nn03873416 划桨\nn03874293 水轮\nn03874599 锁\nn03876231 刷子\nn03877472 睡衣\nn03877845 建筑\nn03884397 乐器\nn03887697 纸巾\nn03888257 降落伞\nn03888605 体操\nn03891251 长椅\nn03891332 停车缴费器\nn03895866 火车 check\nn03899768 院子\nn03902125 公用电话\nn03903868 柱子\nn03908618 文具袋\nn03908714 卷笔刀\nn03916031 香水\nn03920288 培养皿\nn03924679 打印机\nn03929660 吉他拨片\nn03929855 头盔\nn03930313 栅栏\nn03930630 轿车 check\nn03933933 桥\nn03935335 储蓄罐\nn03937543 药丸\nn03938244 枕头\nn03942813 乒乓球\nn03944341 风车\nn03947888 帆船\nn03950228 茶壶\nn03954731 刨子\nn03956157 建筑\nn03958227 塑料袋\nn03961711 碗架\nn03967562 推土机\nn03970156 搋子\nn03976467 相机\nn03976657 杆子\nn03977966 警车 check\nn03980874 披风\nn03982430 桌球\nn03983396 瓶子\nn03991062 盆栽\nn03992509 陶艺\nn03995372 钻机\nn03998194 毯子\nn04004767 打印机\nn04005630 监狱\nn04008634 导弹\nn04009552 投影仪\nn04019541 冰球\nn04023962 拳击\nn04026417 手提袋\nn04033901 羽毛笔\nn04033995 床 被子\nn04037443 赛车\nn04039381 网球\nn04040759 加热器\nn04041544 收音机\nn04044716 卫星接收器\nn04049303 酒桶\nn04065272 房车\nn04067472 鱼竿\nn04069434 相机\nn04070727 冰箱\nn04074963 遥控器\nn04081281 餐厅\nn04086273 手枪\nn04090263 狙击枪\nn04099969 摇椅\nn04111531 烤箱\nn04116512 橡皮\nn04118538 橄榄球\nn04118776 尺\nn04120489 运动鞋\nn04125021 保险箱\nn04127249 回形针\nn04131690 调料瓶\nn04133789 拖鞋\nn04136333 长裙\nn04141076 萨克斯\nn04141327 剑\nn04141975 秤\nn04146614 校车\nn04147183 帆船\nn04149813 计分板\nn04152593 显示器\nn04153751 螺丝\nn04154565 螺丝刀\nn04162706 安全带\nn04179913 缝纫机\nn04192698 盾牌\nn04200800 鞋店\nn04201297 榻榻米\nn04204238 购物篮\nn04204347 购物车\nn04208210 铲子\nn04209133 浴帽\nn04209239 浴帘\nn04228054 滑雪\nn04229816 面罩\nn04235860 睡袋\nn04238763 游标卡尺\nn04239074 移门\nn04243546 老虎机\nn04251144 游泳眼镜\nn04252077 滑雪车\nn04252225 铲雪车\nn04254120 洗手液\nn04254680 足球\nn04254777 袜子\nn04258138 太阳能板\nn04259630 帽子\nn04263257 碗\nn04264628 键盘\nn04265275 电热器\nn04266014 航天飞船\nn04270147 锅铲\nn04273569 快艇\nn04275548 蜘蛛网\nn04277352 毛线\nn04285008 运动型轿车 check\nn04286575 探照灯\nn04296562 乐队\nn04310018 蒸汽机车\nn04311004 桥\nn04311174 鼓\nn04317175 听诊器\nn04325704 担架\nn04326547 石堆\nn04328186 秒表\nn04330267 火炉\nn04332243 滤网\nn04335435 公交车 check\nn04336792 担架\nn04344873 沙发\nn04346328 皇宫\nn04347754 轮船\nn04350905 西装\nn04355338 日晷\nn04355933 墨镜\nn04356056 墨镜\nn04357314 防晒霜\nn04366367 桥\nn04367480 拖把\nn04370456 连帽衫\nn04371430 沙滩裤\nn04371774 秋千\nn04372370 开关\nn04376876 针筒\nn04380533 台灯\nn04389033 坦克\nn04392985 磁带播放器\nn04398044 茶壶\nn04399382 毛绒玩具\nn04404412 电视机\nn04409515 网球\nn04417672 草屋\nn04418357 幕布\nn04423845 指套\nn04428191 装甲车\nn04429376 皇位\nn04435653 瓦片\nn04442312 面包机\nn04443257 烟酒店\nn04447861 马桶\nn04456115 火炬\nn04458633 图腾\nn04461696 大卡车 check\nn04462240 玩具店\nn04465501 拖拉机\nn04467665 大货车 check\nn04476259 碟子 盘子\nn04479046 风衣\nn04482393 儿童自行车\nn04483307 船\nn04485082 三脚架\nn04486054 拱门\nn04487081 巴士 check\nn04487394 长号\nn04493381 浴缸\nn04501370 闸机\nn04505470 打字机\nn04507155 伞\nn04509417 独轮车\nn04515003 钢琴\nn04517823 吸尘器\nn04522168 花瓶\nn04523525 拱廊\nn04525038 珊瑚绒\nn04525305 自动贩卖机\nn04532106 教皇袍\nn04532670 桥\nn04536866 小提琴\nn04540053 排球\nn04542943 煎饼锅\nn04548280 挂钟\nn04548362 钱夹\nn04550184 柜子\nn04552348 飞机\nn04553703 台盆\nn04554684 洗衣机\nn04557648 水瓶\nn04560804 水壶\nn04562935 煤气包\nn04579145 水壶\nn04579432 哨子\nn04584207 头发\nn04589890 窗户\nn04590129 百叶窗\nn04591157 领带\nn04591713 葡萄酒\nn04592741 飞机\nn04596742 炒锅\nn04597913 勺子\nn04599235 围巾\nn04604644 栅栏\nn04606251 沉船\nn04612504 帆船\nn04613696 蒙古包\nn06359193 网页\nn06596364 海报\nn06785654 填字游戏\nn06794110 交通标志\nn06874185 交通灯\nn07248320 书\nn07565083 菜单\nn07579787 菜\nn07583066 菜\nn07584110 菜\nn07590611 菜\nn07613480 蛋糕\nn07614500 冰激凌\nn07615774 棒冰\nn07684084 面包\nn07693725 甜甜圈\nn07695742 面包\nn07697313 汉堡\nn07697537 热狗\nn07711569 焗饭\nn07714571 蔬菜\nn07714990 西蓝花\nn07715103 花椰菜\nn07716358 蔬菜\nn07716906 金瓜\nn07717410 南瓜\nn07717556 南瓜\nn07718472 黄瓜\nn07718747 蔬菜\nn07720875 青椒 黄椒 红椒\nn07730033 花\nn07734744 蘑菇\nn07742313 苹果\nn07745940 草莓\nn07747607 橙子\nn07749582 柠檬\nn07753113 水果\nn07753275 菠萝\nn07753592 香蕉\nn07754684 榴莲\nn07760859 水果\nn07768694 石榴\nn07802026 草垛\nn07831146 意大利面\nn07836838 甜品\nn07860988 面团\nn07871810 肉酱\nn07873807 披萨\nn07875152 派\nn07880968 肉卷\nn07892512 红酒\nn07920052 咖啡\nn07930864 茶\nn07932039 饮料杯\nn09193705 雪山\nn09229709 泡泡\nn09246464 悬崖\nn09256479 珊瑚\nn09288635 温泉\nn09332890 风景\nn09399592 小岛\nn09421951 沙滩\nn09428293 海滩\nn09468604 瀑布 溪流\nn09472597 火山\nn09835506 棒球\nn10148035 婚礼\nn10565667 潜水\nn11879895 油菜花\nn11939491 菊花\nn12057211 植物\nn12144580 玉米\nn12267677 松果\nn12620546 植物\nn12768682 栗子\nn12985857 菌菇\nn12998815 菌菇\nn13037406 菌菇\nn13040303 菌菇\nn13044778 菌菇\nn13052670 菌菇\nn13054560 菌菇\nn13133613 玉米\nn15075141 卷筒纸'
pretrain_list = pretrain_list.split('\n')
pretrain_bool = np.array([])
for i in pretrain_list:
    if (i[-1] == '狗' or  i[-1] == '猫'):
        pretrain_bool = np.append(pretrain_bool, True)
    else:
        pretrain_bool = np.append(pretrain_bool, False)

In [ ]:
pretrain_bool

In [ ]:
#本函数为找到检测出的其他物体大于狗的图像
def not_dog_cat(folder):
    print(folder)
    since = time.time()
    list_nodog = {}
    for pic in os.listdir(folder):
        img = Image.open('{}\\{}'.format(folder, pic))
        img = detect_transforms(img)
        img = img.unsqueeze(0)
        outputs = detect(model_detect, img, single=True)
        max_array = np.where(outputs[0] == np.max(outputs[0], axis=0))
#         print(np.max(outputs, axis=0))
        max_array = max_array[0].astype(int)#找出最大值对应的1000个结果的位置。
#         print(max_array)
        dog_or_cat = list(np.array(pretrain_bool[i] for i in max_array).sum())[0]#对于最大值中，没有哪一个对应的是狗的，将其筛选出来
        if not dog_or_cat:
            list_nodog[pic] = np.max(outputs[0], axis=0)
            print(list_nodog[pic])
    since = time.time() - since
    print('Using {}s'.format(int(since)))
    return list_nodog

以下操作将耗时三十分钟左右

In [ ]:
dict_no_cat = not_dog_cat('train2\\cat')
dict_no_dog = not_dog_cat('train2\\dog')
dict_no_test = not_dog_cat('test')

In [ ]:
#本函数用于移除train2里的symbolic link并且打印不好的图片。
def remove_bad_boy(dict_name, folder):
    img_dict = {k:v for k, v in dict_name.items() if v >=15}
    print('Bad images are these:', '\n', img_dict.keys())
    for img in img_dict.keys():
        if os.path.exists(folder + '\\' + img):
            image = mpimg.imread(folder + '\\' + img)
            plt.figure()
            plt.title(img)
            plt.imshow(image)
            if folder == 'train2\\cat' or folder == 'train2\\dog':#如果是train2里的symbolic link，则需要移除。
                os.remove(folder + '\\' + img)
                print('The bad {} in {} removed.'.format(img, folder))
            else:
                print('The bad {} in {} shouldn\'t be removed'.format(img, folder))
        else:
            print('The bad {} in {} removed.'.format(img, folder))

In [ ]:
remove_bad_boy(dict_no_cat, 'train2\\cat')

In [ ]:
remove_bad_boy(dict_no_dog, 'train2\\dog')

In [ ]:
remove_bad_boy(dict_no_test, 'test')